In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
try:
    os.chdir("./drive/My Drive/RuCode3/")
except Exception as e:
    print(e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import tensorflow as tf

# if tf.__version__ != "1.15.0":
#     !pip install tensorflow==1.15

In [ ]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
from tf_recsys.tfcf.metrics import mae, rmse
from tf_recsys.tfcf.config import Config
from tf_recsys.tfcf.models.svd import SVD
from tf_recsys.tfcf.models.svdpp import SVDPP
from sklearn.model_selection import train_test_split
import pandas as pd
import random

np.random.seed(42)
random.seed(42)
tf.set_random_seed(42)
 
ratings = pd.read_csv("user_ratings.csv", dtype=np.int32)
ratings = ratings.loc[ratings["rating"] != 0]
x = ratings[["user_id", "anime_id"]].values
y = ratings["rating"].values
 
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=0)
 
config = Config()
config.num_users = np.max(x[:, 0]) + 1
config.num_items = np.max(x[:, 1]) + 1
config.min_value = np.min(y)
config.max_value = np.max(y)

epochs = 4
batch_size = 2048
 
with tf.Session() as sess:
    tf.set_random_seed(42)
    model = SVDPP(config, sess, dual=True)

    print("Start train")
    model.train(x_train,
                y_train,
                validation_data=(x_test, y_test),
                epochs=epochs,
                batch_size=batch_size)
 
    y_pred = model.predict(x_test)
    print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))
 
    model.save_model(f'model_{epochs}it_{batch_size}/')
 
    sample = pd.read_csv("sample_submission.csv")
    sample_new = np.array([[int(i) for i in id.split()] for id, rat in sample.values])
    test_preds = model.predict(sample_new)
    sample["rating"] = pd.Series([float(x) for x in test_preds])
    sample.to_csv(f"tfcf_sub_model_{epochs}it_{batch_size}.csv", index=None)

TensorFlow 1.x selected.
Start train



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Epoch 1/4
4793/4793 [==============================] - 6850s 1s/step - rmse: 1.2378 - mae: 0.9260 - val_rmse: 1.1994 - val_mae: 0.8954
Epoch 2/4
4793/4793 [==============================] - 6756s 1s/step - rmse: 1.1511 - mae: 0.8554 - val_rmse: 1.1768 - val_mae: 0.8757
Epoch 3/4
Epoch 3/4
4793/4793 [==============================] - 6795s 1s/step - rmse: 1.1202 - mae: 0.8315 - val_rmse: 1.1725 - val_mae: 0.8734
Epoch 4/4
Epoch 4/4
4793/4793 [==============================] - 6766s 1s/step - rmse: 1.1055 - mae: 0.8206 - val_rmse: 1.1716 - val_mae: 0.8716
rmse: 1.1716089582704807, mae: 0.8715814342568143

rmse: 1.1716089582704807, mae: 0.8715814342568143

